In [1]:
# this program will be what I run almost everyday in order to see if there are any big changes in OI for any tickers.






In [2]:
import os
import sys
import requests

import time

# need to see processing speed
from time import process_time

# need for date functions
from pandas.tseries.offsets import DateOffset


from pandas import DataFrame
import pandas as pd

import numpy as np

import re

# bold text
from IPython.display import Markdown, display
def printmd(string):
    display(Markdown(string))

# styles pandas dataframes
from IPython.core.display import HTML
css = open('style-table.css').read() + open('style-notebook.css').read()
HTML('<style>{}</style>'.format(css))

# need for my date functions
from datetime import date


import pandas as pd
import numpy as np
import chart_studio.plotly as py
import cufflinks as cf
import seaborn as sns   #used for the datasets provided by seaborn
import plotly.express as px
%matplotlib inline
import plotly.graph_objects as go



from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)
cf.go_offline()

In [3]:
holidays2020 = ['2020-01-01', '2020-01-20', '2020-02-17', '2020-04-10', '2020-05-25', '2020-07-03', '2020-09-07']
holidays2021 = ['2021-01-01', '2021-01-18', '2021-02-15', '2021-04-02', '2021-05-31', '2021-07-05', '2021-09-06', '2021-11-25', '2021-12-24']
holidays2022 = ['2022-01-17', '2022-02-21', '2022-04-15', '2022-05-30', '2022-06-20', '2022-07-04', '2022-09-05', '2022-12-26']

weekdays_year = pd.date_range(start='2022/1/1', end='2022/12/31', freq='B')

# -- END -- #






def closestTradingDate():
    # closest trading day
    from pandas.tseries.offsets import DateOffset

    today_date = pd.Timestamp(date.today())
    closest_date = today_date

    # Checks if today's date is SATURDAY
    if (today_date.isoweekday() == 6):
        closest_date = today_date - DateOffset(days = 1)
    # Checks if today_date is SUNDAY

    elif (today_date.isoweekday() == 7):
        closest_date = today_date - DateOffset(days = 2)

    # Check if the Friday is a holiday and subtract another day
    while(str(closest_date)[:10] in holidays2022):
        closest_date = closest_date - DateOffset(days = 1)
    
    # returns only date ('YYYY-MM-DD') without time
    closest_date = str(closest_date)[:10]
    
    return closest_date


def listOfDatesToCurrent():
# returns a list of active trading dates from the year 2022 (can change to be any year)
    dates = []

    for num in range(len(weekdays_year)):

        # Changing it from a 'pandas timestamp' to a string with only year-month-day
        dates.append(weekdays_year.date[num].strftime('%Y-%m-%d'))

    # Disclude holidays from active trading days
    dates = [date for date in dates if date not in holidays2022]
    try:
        # this slice will stop at today's date from first day
        # finds the index of today's date (date.today()) and slices list to stop at today
        dates_to_present = dates[:dates.index(closestTradingDate())+1]
    except:
        print("ERROR?! UNCOMMENT BELOW LINE TO GIVE STATIC AMOUNT OF DATES")
    #     dates_to_present = dates[:134]

    return dates_to_present


In [4]:
def dateRange_filepathList(ticker, date1, date2):
    datelist = list(reversed(listOfDatesToCurrent()))
    if datelist.index(date1) > datelist.index(date2):
        temp = date1
        date1 = date2
        date2 = temp
    return [# list comprehension that returns a list of file paths between 2 dates
            r"C:\Users\lorig\Desktop\PythonCode\Projects, Exercises, Programs\Trade Alert Journey\Data\{}\{}_intraday_volume\{}_intraday_volume_{}.csv"
            .format(ticker, ticker, ticker, datelist[i]) for i in range(datelist.index(date1), datelist.index(date2)+1)
           ]

What the logic should be:

Get total calls & puts for 1 day (starting from closest day --> further)




In [6]:
date1 = "2022-08-01"
date2 = "2022-09-16"

datafolder_path = r"C:\Users\lorig\Desktop\PythonCode\Projects, Exercises, Programs\Trade Alert Journey\Data"

for ticker in os.listdir(datafolder_path):
    # ** OVERRIDE don't forget to remove break at end
#     ticker = "XLE"
    # ***
    
    try:
        counter = 0

        avecalls = 0
        aveputs = 0
        ave5calls = []
        ave5puts = []
        ave17calls = []
        ave17puts = []
        ave30calls = []
        ave30puts = []

        fileList = dateRange_filepathList(ticker, date1, date2)
        for file in fileList:
            df = pd.read_csv(file)
            
            # this will only return average sum_calls and puts FOR ONE DAY
            avecalls = df.sum_calls.max()#/len(df.sum_calls)
            aveputs = df.sum_puts.max()#/len(df.sum_puts)
#             print(file[-14:-4], "calls", avecalls, "...puts:", aveputs )

            if counter < 5:
                ave5calls.append(int(avecalls))
                ave5puts.append(int(aveputs))

            if counter < 17:
                ave17calls.append(int(avecalls))
                ave17puts.append(int(aveputs))

            if counter < 30:
                ave30calls.append(int(avecalls))
                ave30puts.append(int(aveputs))

            counter += 1

        if len(fileList) < 5:
            ave5calls = sum(ave5calls)/len(fileList)
            ave5puts = sum(ave5puts)/len(fileList)
        else:
            ave5calls = sum(ave5calls)/5
            ave5puts = sum(ave5puts)/5

#         print(ticker, "\n")

#         print("Ave 5 calls:", ave5calls)
    
        if len(fileList) < 17:
            ave17calls = sum(ave17calls)/len(fileList)
            ave17puts = sum(ave17puts)/len(fileList)
            
        else:
            ave17calls = sum(ave17calls)/17
            ave17puts = sum(ave17puts)/17

#         print("Ave 17 calls:", ave17calls)

        if len(fileList) < 30:
            ave30calls = sum(ave30calls)/len(fileList)
            ave30puts = sum(ave30puts)/len(fileList)
        else:
            ave30calls = sum(ave30calls)/30
            ave30puts = sum(ave30puts)/30

#         print("Ave 30 calls:", ave30calls)
#         print()

#         print("Ave 5 puts:", ave5puts)

#         print("Ave 17 puts:", ave17puts)

#         print("Ave 30 puts:", ave30puts)
#         print()
        # at this point, haven't skipped to next ticker yet.

        # score1 == 5day average / 17day
        score1calls = ave5calls / ave17calls
        score1puts = ave5puts / ave17puts

        # score2 == 5day average / 30 day average
        score2calls = ave5calls / ave30calls
        score2puts = ave5puts / ave30puts

#         print("**5day/17day", "\ncalls:", score1calls, "\nputs", score1puts)

#         print("**5day/30day", "\ncalls:", score2calls, "\nputs", score2puts)

#         print("\n\n\n")

        # AT THIS POINT I NOW HAVE 5,17,30 DAY AVERAGES FOR CALL&PUT VOLUMES
        # NOW I NEED TO SEE IF THERE IS A **DRAMATIC** CHANGE IN VOLUMES
        # I WILL SCORE THIS IN TERMS OF FRACTIONS GREATER THAN.
        # IF FOR EXAMPLE VERU HAS 30DAY: 15K CALL VOLUME & 25K PUT VOLUME
        # IF 5DAY CALL VOLUME --> 35K ... THAT MEANS 35K/15K = 2.33


        # NOW I NEED TO WRITE SOME CONDITIONS TO SEPERATE THE BIG CHANGE TICKERS
        # AKA if my 'score1' > 3 this means there is 3X the 17day average volume
        # if 'score2' > 5 this means there is 5x the 30day voume
        # if score2 == 0.5, this means there is 50% LESS volume than 30day average
        displayedState = False
        if score2calls > 3:
    #         printmd("**5-Day VS. 30-day**")
            displayedState = True
            print("{} 5-day calls currently has {}X the 30-day volume. ".format(ticker, score2calls))

        if score1calls > 2:
    #         printmd("**5-Day VS. 30-day**")
            displayedState = True

            print("{} 5-day calls currently has {}X the 17-day volume. ".format(ticker, score1calls))

        if score2puts > 3:
            displayedState = True

            print("{} 5-day puts currently has {}X the 30-day volume. ".format(ticker, score2puts))
        if score1puts > 2:
    #         printmd("**5-Day VS. 30-day**")
            displayedState = True

            print("{} 5-day puts currently has {}X the 17-day volume. ".format(ticker, score1puts))
#         break
        if displayedState == True: print("length of file selection", len(fileList), "------------------------------------------------------\n")

    except:
        continue


ADBE 5-day calls currently has 2.0275611538626883X the 17-day volume. 
length of file selection 34 ------------------------------------------------------

ALNY 5-day calls currently has 2.0101727957465663X the 17-day volume. 
length of file selection 34 ------------------------------------------------------

ALTO 5-day puts currently has 2.110526315789474X the 17-day volume. 
length of file selection 34 ------------------------------------------------------

ARCC 5-day calls currently has 3.262579797221179X the 30-day volume. 
ARCC 5-day calls currently has 2.349249458218184X the 17-day volume. 
length of file selection 34 ------------------------------------------------------

ASTR 5-day puts currently has 3.2057357158614606X the 17-day volume. 
length of file selection 34 ------------------------------------------------------

ATUS 5-day calls currently has 3.7112536335532686X the 30-day volume. 
ATUS 5-day calls currently has 2.9349173642412536X the 17-day volume. 
ATUS 5-day puts c

METV 5-day calls currently has 2.4943711284675465X the 17-day volume. 
length of file selection 34 ------------------------------------------------------

MNTV 5-day calls currently has 2.943673959299471X the 17-day volume. 
MNTV 5-day puts currently has 2.544869215291751X the 17-day volume. 
length of file selection 34 ------------------------------------------------------

NLY 5-day calls currently has 3.05486488038193X the 30-day volume. 
NLY 5-day calls currently has 2.545286302419512X the 17-day volume. 
length of file selection 34 ------------------------------------------------------

NMM 5-day calls currently has 2.675236806495264X the 17-day volume. 
length of file selection 34 ------------------------------------------------------

NRG 5-day calls currently has 3.371367939928175X the 30-day volume. 
NRG 5-day calls currently has 2.781108162699513X the 17-day volume. 
length of file selection 34 ------------------------------------------------------

OHI 5-day calls currently 